# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
vacation_data = "../output_data/cities.csv"
vacation_data_df = pd.read_csv(vacation_data)

vacation_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bengkulu,-3.80,102.27,75.74,92,62,5.01,ID,1597096132
1,sao joao da barra,-21.64,-41.05,73.40,78,1,16.11,BR,1597095985
2,douglas,41.30,-96.17,84.99,45,40,6.93,US,1597096133
3,nuevo progreso,23.17,-97.95,87.01,58,90,7.00,MX,1597096133
4,mahebourg,-20.41,57.70,64.40,77,40,8.93,MU,1597095892


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
location = vacation_data_df[["Lat", "Lng"]]
humidity = vacation_data_df["Humidity"]

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humidity, dissipating=False, max_intensity=250, point_radius=3)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
best_clouds = vacation_data_df.loc[(vacation_data_df["Cloudiness"] > 50)&
                                   (vacation_data_df["Cloudiness"] < 80)]

best_temp = vacation_data_df.loc[(vacation_data_df["Max Temp"] > 50)&
                                   (vacation_data_df["Max Temp"] < 80)]

low_humidity = vacation_data_df.loc[(vacation_data_df["Humidity"] > 10)&
                                   (vacation_data_df["Humidity"] < 20)]

low_humidity

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,elko,41.00,-115.50,93.20,12,1,6.93,US,1597096141
142,san ramon,37.78,-121.98,91.40,17,1,14.99,US,1597095934
159,riyadh,24.69,46.72,89.60,15,65,8.05,SA,1597096023
363,atar,20.52,-13.05,101.82,19,99,2.62,MR,1597096243
407,riverton,40.52,-111.94,95.00,18,1,13.87,US,1597096190
484,fallon,39.47,-118.78,96.01,12,40,10.29,US,1597096286
495,deming,32.27,-107.76,102.20,12,1,24.16,US,1597096290
529,sabha,27.04,14.43,91.92,18,70,10.96,LY,1597096301
534,taft,31.75,54.21,93.20,17,2,9.17,IR,1597096302


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = low_humidity 

hotel_df["Hotel Name"] = ""
hotel_df

C:\Users\trevo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
36,elko,41.00,-115.50,93.20,12,1,6.93,US,1597096141,
142,san ramon,37.78,-121.98,91.40,17,1,14.99,US,1597095934,
159,riyadh,24.69,46.72,89.60,15,65,8.05,SA,1597096023,
363,atar,20.52,-13.05,101.82,19,99,2.62,MR,1597096243,
407,riverton,40.52,-111.94,95.00,18,1,13.87,US,1597096190,
484,fallon,39.47,-118.78,96.01,12,40,10.29,US,1597096286,
495,deming,32.27,-107.76,102.20,12,1,24.16,US,1597096290,
529,sabha,27.04,14.43,91.92,18,70,10.96,LY,1597096301,
534,taft,31.75,54.21,93.20,17,2,9.17,IR,1597096302,


In [8]:
target_search = "hotel"
target_radius = 5000
target_type = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Lng"] 

    params['location'] = f"{lat},{lng}"

    print(f"Retrieving Results for Index {index}: {row['City']}.")
    print("-" *20)
    response = requests.get(base_url, params=params).json()
    
    results = response['results']
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Retrieving Results for Index 36: elko.
--------------------
Missing field/result... skipping.
------------
Retrieving Results for Index 142: san ramon.
--------------------
Closest hotel is San Ramon Marriott.
------------
Retrieving Results for Index 159: riyadh.
--------------------


C:\Users\trevo\Anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Closest hotel is Four Seasons Hotel Riyadh At Kingdom Center.
------------
Retrieving Results for Index 363: atar.
--------------------
Closest hotel is Odar kanawal.
------------
Retrieving Results for Index 407: riverton.
--------------------
Closest hotel is Embassy Suites by Hilton South Jordan Salt Lake City.
------------
Retrieving Results for Index 484: fallon.
--------------------
Closest hotel is Holiday Inn Express Fallon.
------------
Retrieving Results for Index 495: deming.
--------------------
Closest hotel is Holiday Inn Express & Suites Deming.
------------
Retrieving Results for Index 529: sabha.
--------------------
Closest hotel is فندق علي.
------------
Retrieving Results for Index 534: taft.
--------------------
Closest hotel is Shirkooh hotel Taft.
------------


In [9]:
hotel_df = hotel_df[hotel_df["Hotel Name"]!= ""]

hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
142,san ramon,37.78,-121.98,91.40,17,1,14.99,US,1597095934,San Ramon Marriott
159,riyadh,24.69,46.72,89.60,15,65,8.05,SA,1597096023,Four Seasons Hotel Riyadh At Kingdom Center
363,atar,20.52,-13.05,101.82,19,99,2.62,MR,1597096243,Odar kanawal
407,riverton,40.52,-111.94,95.00,18,1,13.87,US,1597096190,Embassy Suites by Hilton South Jordan Salt Lak...
484,fallon,39.47,-118.78,96.01,12,40,10.29,US,1597096286,Holiday Inn Express Fallon
495,deming,32.27,-107.76,102.20,12,1,24.16,US,1597096290,Holiday Inn Express & Suites Deming
529,sabha,27.04,14.43,91.92,18,70,10.96,LY,1597096301,فندق علي
534,taft,31.75,54.21,93.20,17,2,9.17,IR,1597096302,Shirkooh hotel Taft


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
